# First Frontier LLM Project

## Import the necessary libraries

In [1]:
import os
from openai import OpenAI
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
current_path = os.getcwd()
grandparent = os.path.dirname(os.path.dirname(current_path))
grandparent

'/mnt/e/Development/Python/LLMEngineering/LLMEngineeringPractice'

In [3]:
# Load environment variabls
env_path = grandparent + "/.env"
load_dotenv(env_path,override=True)
api_key = os.getenv('OPENAI_API_KEY')

# check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


## Call OpenAI API to generate text

In [4]:
openai = OpenAI()

message = "Hello, my name is Duy Khang. This is my first ever message to you! Nice to meet you!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello Duy Khang! It’s great to meet you too! How can I assist you today?


*The role of the Language Model (LM) in a Large Language Model (LLM) is to generate human-like text based on the input it receives. It does this by predicting the most likely next word (or token) in a sequence, using probabilities learned from vast amounts of training data.*

### Some of interesting in data crawling

In [9]:
headers = {
    "User-Agent" :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title can be found!"
        for irr in soup.body(["script", "style", "img", "input"]):
            irr.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
ex = Website("https://edwarddonner.com")
print(ex.title)
print(ex.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts
There are two main types of prompts used in LLMs:
1. **System Prompts**: These prompts set the behavior and context for the model. They are typically used to define the role of the model, such as "You are a helpful assistant." System prompts help guide the model's responses to be more aligned with the desired output. That tells them what.task the are performing and what tone they should use.
2. **User Prompts**: These prompts are the actual questions or instructions given to the model by the user. They can be specific queries, requests for information, or any other input that the user wants the model to respond to. The conversation starter that they should reply to.

In [11]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
def user_prompt_for(website):
    if website is None:
        raise ValueError()
    
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\n The contents of this website is as follows; \
        please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
print(user_prompt_for(ex))

You are looking at a website titled Home - Edward Donner
 The contents of this website is as follows;         please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup unta

## Message structure
The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

> [
>    {"role": "system", "content": "system message goes here"},

>    {"role": "user", "content": "user message goes here"}
>]
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [19]:
def messages_for(website):

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website=website)}
    ]

In [20]:
messages_for(ex)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\n The contents of this website is as follows;         please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it c

## Bringing it all together

In [22]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website=website)
    )
    return response.choices[0].message.content

In [26]:
url = "https://anthropic.com"
summarize(url)

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [28]:
display_summary("https://edwarddonner.com")

# Website Summary: Edward Donner

Edward Donner's website serves as a personal and professional platform that showcases his interests in coding, experimenting with large language models (LLMs), and electronic music. He introduces himself as the co-founder and CTO of Nebula.io, a company focused on using AI to enhance talent discovery and engagement.

## Key Highlights:
- Ed shares his passion for writing code, LLMs, and DJing, albeit mentioning he is out of practice in music production.
- Nebula.io pioneers in applying AI in talent management, boasting a patented matching model and notable press coverage.
- Ed is previously the founder and CEO of an AI startup, untapt, which was acquired in 2021.

## Recent Announcements:
1. **September 15, 2025** - Discussion on "AI in Production: Gen AI and Agentic AI on AWS at scale."
2. **May 28, 2025** - Coverage on connecting his courses to help others become experts in LLMs.
3. **May 18, 2025** - Insights shared from the "2025 AI Executive Briefing."
4. **April 21, 2025** - Launch of "The Complete Agentic AI Engineering Course."

Overall, the website reflects Ed's commitment to advancing AI and LLM technologies while connecting with like-minded individuals.

In [ ]:
# Exercise

# 1. Create the system prompt
system_prompt = (
    "You are an AI assistant specialized in summarization for business communication. "
    "Your answers must be concise, professional, and highlight key facts. "
    "Always preserve important numeric data in the summary. "
    "Respond in a way that could be directly used in a business context."
)


# 2. Example user prompt: summarization for business 
user_prompt = """
Here is the email body:
"Dear team, I'm excited to share that our Q3 sales increased by 35% compared to last quarter, 
with the strongest growth in our e-commerce channel. Let's discuss next week how we can build 
on this momentum."

Generate a short subject line for this email.
"""

# 3. Put into message format
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]


# from openai import OpenAI
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)

Subject: Q3 Sales Up 35% - Let's Build on Momentum!


In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def scrape_with_selenium(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    # Tự động tải chromedriver đúng version
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    driver.quit()
    return soup.get_text()

print(scrape_with_selenium("https://openai.com"))


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x60375399d4e3 <unknown>
#1 0x6037536ccc76 <unknown>
#2 0x6037536f3757 <unknown>
#3 0x6037536f2029 <unknown>
#4 0x603753730ccc <unknown>
#5 0x60375373047f <unknown>
#6 0x603753727de3 <unknown>
#7 0x6037536fd2dd <unknown>
#8 0x6037536fe34e <unknown>
#9 0x60375395d3e4 <unknown>
#10 0x6037539613d7 <unknown>
#11 0x60375396bb20 <unknown>
#12 0x603753962023 <unknown>
#13 0x6037539301aa <unknown>
#14 0x6037539866b8 <unknown>
#15 0x603753986847 <unknown>
#16 0x603753996243 <unknown>
#17 0x74dd28294ac3 <unknown>
